In [1]:
from itertools import chain, combinations, product

def powerset(iterable):
    """
    powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)
    """
    xs = list(iterable)
    # note we return an iterator rather than a list
    return chain.from_iterable(combinations(xs,n) for n in range(len(xs)+1))

def is_vi_satisfied(collection_of_neighborhood_systems, n):
    for x in range(1, n + 1):
        for v in collection_of_neighborhood_systems[x]:
            if not(x in v):
                return False
    return True

def is_vii_satisfied(collection_of_neighborhood_systems, powset_minus_empty_set_and_S, n):
    for x in range(1, n + 1):
        for v in collection_of_neighborhood_systems[x]:
            for v_ in powset_minus_empty_set_and_S:
                if set(v) <= set(v_):
                    if not(v_ in collection_of_neighborhood_systems[x]):
                        return False
    return True

def is_viii_satisfied(collection_of_neighborhood_systems, n):
    for x in range(1, n + 1):
        for v1, v2 in combinations(collection_of_neighborhood_systems[x], 2):
            if not(tuple(sorted(set(v1) & set(v2))) in collection_of_neighborhood_systems[x]):
                return False
    return True

def is_viv_satisfied(collection_of_neighborhood_systems, n):
    for x in range(1, n + 1):
        for v in collection_of_neighborhood_systems[x]:
            for w in collection_of_neighborhood_systems[x]:
                flag = True
                for y in w:
                    if not(v in collection_of_neighborhood_systems[y]):
                        flag = False
                        break
                if flag:
                    break
            if not(flag):
                return False
    return True
    
def all_collections_of_neighborhood_systems(n):
    """
    S = {1, 2, ..., n}
    """
    powset_of_S = tuple(powerset(range(1, n + 1)))
    powset_of_S = tuple(tuple(sorted(t)) for t in powset_of_S)
    S = tuple(range(1, n + 1))
    powset_minus_empty_set_and_S = tuple(s for s in powset_of_S if s != () and s != S)
    list_of_collections_of_neighborhood_systems = []

    for t in product(powerset(powset_minus_empty_set_and_S), repeat = n):
        collection_of_neighborhood_systems = {}
        for i in range(1, n + 1):
            collection_of_neighborhood_systems[i] = t[i - 1]
        if not(is_vi_satisfied(collection_of_neighborhood_systems, n)):
            continue
        if not(is_vii_satisfied(collection_of_neighborhood_systems, powset_minus_empty_set_and_S, n)):
            continue
        if not(is_viii_satisfied(collection_of_neighborhood_systems, n)):
            continue
        if not(is_viv_satisfied(collection_of_neighborhood_systems, n)):
            continue
        collection_of_neighborhood_systems = {x:(collection_of_neighborhood_systems[x] + (S,)) for x in collection_of_neighborhood_systems.keys()}
        list_of_collections_of_neighborhood_systems.append(collection_of_neighborhood_systems)
    return list_of_collections_of_neighborhood_systems

In [2]:
l = all_collections_of_neighborhood_systems(3)

print("All Collections of Neighborhood Systems of {1, 2, 3}:")
print()
for i in range(len(l)):
    if i == 0:
        print(str(i+1) + "st Collection of Neighborhood Systems:")
    if i == 1:
        print(str(i+1) + "nd Collection of Neighborhood Systems:")
    if i == 2:
        print(str(i+1) + "rd Collection of Neighborhood Systems:")
    if i >= 3:
        print(str(i+1) + "th Collection of Neighborhood Systems:")
    print(l[i])
    print()

All Collections of Neighborhood Systems of {1, 2, 3}:

1st Collection of Neighborhood Systems:
{1: ((1, 2, 3),), 2: ((1, 2, 3),), 3: ((1, 2, 3),)}

2nd Collection of Neighborhood Systems:
{1: ((1, 2, 3),), 2: ((1, 2, 3),), 3: ((3,), (1, 3), (2, 3), (1, 2, 3))}

3rd Collection of Neighborhood Systems:
{1: ((1, 2, 3),), 2: ((2, 3), (1, 2, 3)), 3: ((2, 3), (1, 2, 3))}

4th Collection of Neighborhood Systems:
{1: ((1, 2, 3),), 2: ((2, 3), (1, 2, 3)), 3: ((3,), (1, 3), (2, 3), (1, 2, 3))}

5th Collection of Neighborhood Systems:
{1: ((1, 2, 3),), 2: ((2,), (1, 2), (2, 3), (1, 2, 3)), 3: ((1, 2, 3),)}

6th Collection of Neighborhood Systems:
{1: ((1, 2, 3),), 2: ((2,), (1, 2), (2, 3), (1, 2, 3)), 3: ((2, 3), (1, 2, 3))}

7th Collection of Neighborhood Systems:
{1: ((1, 2, 3),), 2: ((2,), (1, 2), (2, 3), (1, 2, 3)), 3: ((3,), (1, 3), (2, 3), (1, 2, 3))}

8th Collection of Neighborhood Systems:
{1: ((1, 2), (1, 2, 3)), 2: ((1, 2), (1, 2, 3)), 3: ((1, 2, 3),)}

9th Collection of Neighborhood Sy